# 1. Miscellaneous techniques

## 1.1. Mapping
Mapping in PySpark requires using the `udf()` function, which allows a Python function to work on PySpark dataframes.

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [8]:
cars = pd.read_excel(r'data\cars.xlsx')
cars = spark.createDataFrame(cars.astype(str)).replace('nan', None)
cars = cars.withColumn('price', F.col('price').cast('float'))
cars.limit(5)

manufacturer,model,type,min_price,price,max_price,mpg_city,mpg_highway,airbags,drive_train,cylinders,engine_size,horsepower,rpm,rev_per_mile,man_trans_avail,fuel_tank_capacity,passengers,length,wheelbase,width,turn_circle,rear_seat_room,luggage_room,weight,origin,make
Chevrolet,Cavalier,Compact,8.5,13.4,18.3,25,36,None,Front,4.0,2.2,110,5200,2380,Yes,15.2,5,182,101,66,38,25.0,13.0,2490,USA,Chevrolet Cavalier
Chevrolet,Corsica,Compact,11.4,11.4,11.4,25,34,Driver only,Front,4.0,2.2,110,5200,2665,Yes,15.6,5,184,103,68,39,26.0,14.0,2785,USA,Chevrolet Corsica
Chevrolet,Camaro,Sporty,13.4,15.1,16.8,19,28,Driver & Passenger,Rear,6.0,3.4,160,4600,1805,Yes,15.5,4,193,101,74,43,25.0,13.0,3240,USA,Chevrolet Camaro
Chevrolet,Lumina,Midsize,13.4,15.9,18.4,21,29,None,Front,4.0,2.2,110,5200,2595,No,16.5,6,198,108,71,40,28.5,16.0,3195,USA,Chevrolet Lumina
Chevrolet,Lumina_APV,Van,14.7,16.3,18.0,18,23,None,Front,6.0,3.8,170,4800,1690,No,20.0,7,178,110,74,44,30.5,null,3715,USA,Chevrolet Lumina_APV


In [20]:
cars.approxQuantile('price', [0.25, 0.5, 0.75], relativeError=0)

[11.600000381469727, 15.899999618530273, 18.799999237060547]

In [33]:
def tmp_getPriceLevel(price):
    if price < 11.6:
        return 'very low'
    elif price < 15.9:
        return 'low'
    elif price < 18.8:
        return 'high'
    else:
        return 'very high'

getPriceLevel = F.udf(lambda price: tmp_getPriceLevel(price))

In [40]:
cars.select('price', getPriceLevel('price').alias('price_level')).limit(5)

price,price_level
13.4,low
11.4,very low
15.1,low
15.9,low
16.3,high


## 1.2. Window functions

In [2]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [6]:
data = (
    ('James', 'Sales', 3000),
    ('Harry', 'Sales', 3500),
    ('Ash', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('Wayne', 'Sales', 3000),
    ('Scott', 'Finance', 3300),
    ('Jen', 'Finance', 3900),
    ('Jeff', 'Marketing', 3000),
    ('Kumar', 'Marketing', 2000),
    ('Saif', 'Sales', 4100))
 
columns= ['employee', 'department', 'salary']

df = spark.createDataFrame(data=data, schema=columns)
df

employee,department,salary
James,Sales,3000
Harry,Sales,3500
Ash,Sales,3000
Michael,Sales,4600
Robert,Sales,4100
Maria,Finance,3000
Wayne,Sales,3000
Scott,Finance,3300
Jen,Finance,3900
Jeff,Marketing,3000


In addition to activate window functions, we firstly initialize the window.

In [3]:
from pyspark.sql.window import Window
window = Window.partitionBy('department').orderBy('salary')

#### Ranking

In [7]:
df\
    .withColumn('row_number', F.row_number().over(window))\
    .withColumn('rank', F.rank().over(window))\
    .withColumn('dense_rank', F.dense_rank().over(window))\
    .withColumn('percent_rank', F.round(F.percent_rank().over(window), 2))\
    .withColumn('cume_dist', F.round(F.cume_dist().over(window), 2))\
    .withColumn('ntile_3', F.ntile(3).over(window))

employee,department,salary,row_number,rank,dense_rank,percent_rank,cume_dist,ntile_3
James,Sales,3000,1,1,1,0.0,0.43,1
Ash,Sales,3000,2,1,1,0.0,0.43,1
Wayne,Sales,3000,3,1,1,0.0,0.43,1
Harry,Sales,3500,4,4,2,0.5,0.57,2
Robert,Sales,4100,5,5,3,0.67,0.86,2
Saif,Sales,4100,6,5,3,0.67,0.86,3
Michael,Sales,4600,7,7,4,1.0,1.0,3
Maria,Finance,3000,1,1,1,0.0,0.33,1
Scott,Finance,3300,2,2,2,0.5,0.67,2
Jen,Finance,3900,3,3,3,1.0,1.0,3


#### Shifting

In [38]:
df\
    .withColumn('lag_1', F.lag('salary', 1).over(window))\
    .withColumn('lead_2', F.lead('salary', 2).over(window))

employee,department,salary,lag_1,lead_2
James,Sales,3000,null,3000
Ash,Sales,3000,3000,3500
Wayne,Sales,3000,3000,4100
Harry,Sales,3500,3000,4100
Robert,Sales,4100,3500,4600
Saif,Sales,4100,4100,null
Michael,Sales,4600,4100,null
Maria,Finance,3000,null,3900
Scott,Finance,3300,3000,null
Jen,Finance,3900,3300,null


#### Aggregating

In [47]:
df\
    .withColumn('cumsum', F.sum('salary').over(window))

employee,department,salary,cumsum
James,Sales,3000,9000
Ash,Sales,3000,9000
Wayne,Sales,3000,9000
Harry,Sales,3500,12500
Robert,Sales,4100,20700
Saif,Sales,4100,20700
Michael,Sales,4600,25300
Maria,Finance,3000,3000
Scott,Finance,3300,6300
Jen,Finance,3900,10200


# 2. Pivot table

## 2.1. Unpivot

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [99]:
data = (
    ('red', 1000, 1200, 1500),
    ('green', 1500, 1500, 1575),
    ('blue', 2000, 2200, 2000)
)

columns = ['color', 'small', 'medium', 'large']

df = spark.createDataFrame(data, schema=columns)
df

color,small,medium,large
red,1000,1200,1500
green,1500,1500,1575
blue,2000,2200,2000


In [100]:
df.select('color', F.expr('stack(3, "small", small, "medium", medium, "large", large) as (size, price)'))

color,size,price
red,small,1000
red,medium,1200
red,large,1500
green,small,1500
green,medium,1500
green,large,1575
blue,small,2000
blue,medium,2200
blue,large,2000


## 2.2. Pivot table

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [62]:
supermarket = spark.read.csv(r'data\supermarket_sales.csv', header=True)
supermarket.limit(5)

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,01/05/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic access...,15.28,5,3.82,03/08/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,03/03/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,02/08/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3


In [64]:
supermarket\
    .groupBy('product_line')\
    .pivot('city')\
    .agg(F.round(F.mean('profit'), 2))

product_line,Mandalay,Naypyitaw,Yangon
Home and lifestyle,16.71,14.7,16.42
Fashion accessories,12.61,15.79,15.25
Health and beauty,17.95,15.22,12.76
Electronic access...,14.76,16.42,14.54
Food and beverages,14.49,17.15,14.09
Sports and travel,15.35,16.68,15.64


#### Multivariate pivoting

In [67]:
supermarket\
    .withColumn('info', F.concat(F.col('gender'), F.lit(', '), F.col('customer_type')))\
    .groupBy('product_line')\
    .pivot('info')\
    .agg(F.round(F.mean('profit'), 2))

product_line,"Female, Member","Female, Normal","Male, Member","Male, Normal"
Home and lifestyle,17.46,19.05,14.21,13.84
Fashion accessories,15.32,14.88,13.68,14.03
Health and beauty,13.3,14.26,19.33,13.95
Electronic access...,15.17,15.5,14.78,15.38
Food and beverages,18.3,16.57,13.02,13.03
Sports and travel,15.55,15.34,15.31,16.98


## 2.3. Crosstab
Crosstab is a special case of pivot table, where `count` is selected as the aggregate function.

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [62]:
supermarket = spark.read.csv(r'data\supermarket_sales.csv', header=True)
supermarket.limit(5)

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,01/05/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic access...,15.28,5,3.82,03/08/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,03/03/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,02/08/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3


In [68]:
supermarket.crosstab('city', 'payment')

city_payment,Cash,Credit card,Ewallet
Naypyitaw,124,98,106
Mandalay,110,109,113
Yangon,110,104,126


# 3. Combining datasets

## 3.1. Union
PySpark supports two union methods:
- `union`: union using the current order of columns
- `unionByName`: union using column names

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [102]:
data = (
    (2019, 1, 2500),
    (2019, 2, 3500),
    (2019, 3, 4000),
    (2019, 4, 5000)
)

columns = ['year', 'quarter', 'profit']

df_19 = spark.createDataFrame(data, schema=columns)
df_19

year,quarter,profit
2019,1,2500
2019,2,3500
2019,3,4000
2019,4,5000


In [103]:
data = (
    (2020, 1, 2700),
    (2020, 2, 3900),
    (2020, 3, 5000),
    (2020, 4, 8000)
)

columns = ['year', 'quarter', 'profit']

df_20 = spark.createDataFrame(data, schema=columns)
df_20

year,quarter,profit
2020,1,2700
2020,2,3900
2020,3,5000
2020,4,8000


In [104]:
df_19.union(df_20)

year,quarter,profit
2019,1,2500
2019,2,3500
2019,3,4000
2019,4,5000
2020,1,2700
2020,2,3900
2020,3,5000
2020,4,8000


In [105]:
df_19.unionByName(df_20)

year,quarter,profit
2019,1,2500
2019,2,3500
2019,3,4000
2019,4,5000
2020,1,2700
2020,2,3900
2020,3,5000
2020,4,8000


## 3.2. Join

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

In [125]:
data = (
    ('Hannah', 1200, 'Allowance'),
    ('James', 3000, 'Basic'),
    ('Gabriel', 700, 'Allowance'),
    ('Smith', 2000, 'Basic'),
    ('Alex', 10000, 'Higher'),
)

columns = ['name', 'income_before_tax', 'tax_band']

income = spark.createDataFrame(data, schema=columns)
income

name,income_before_tax,tax_band
Hannah,1200,Allowance
James,3000,Basic
Gabriel,700,Allowance
Smith,2000,Basic
Alex,10000,Higher


In [118]:
data = (
    ('Allowance', 'Up to 12,500', 0.0),
    ('Basic', '12,501 to 50,000', 0.2),
    ('Higher', '50,001 to 150,000', 0.4),
    ('Additional', 'Over 150,000', 0.45),
)

columns = ['band', 'income_range', 'tax_rate']

tax = spark.createDataFrame(data, columns)
tax

band,income_range,tax_rate
Allowance,"Up to 12,500",0.0
Basic,"12,501 to 50,000",0.2
Higher,"50,001 to 150,000",0.4
Additional,"Over 150,000",0.45


In [122]:
income.join(tax, income.tax_band==tax.band, how='left')

name,income_before_tax,tax_band,band,income_range,tax_rate
Hannah,1200,Allowance,Allowance,"Up to 12,500",0.0
Gabriel,700,Allowance,Allowance,"Up to 12,500",0.0
Alex,10000,Higher,Higher,"50,001 to 150,000",0.4
James,3000,Basic,Basic,"12,501 to 50,000",0.2
Smith,2000,Basic,Basic,"12,501 to 50,000",0.2


In [128]:
income\
    .withColumnRenamed('tax_band', 'band')\
    .join(tax, on='band', how='left')

band,name,income_before_tax,income_range,tax_rate
Allowance,Hannah,1200,"Up to 12,500",0.0
Allowance,Gabriel,700,"Up to 12,500",0.0
Higher,Alex,10000,"50,001 to 150,000",0.4
Basic,James,3000,"12,501 to 50,000",0.2
Basic,Smith,2000,"12,501 to 50,000",0.2


#### Cross join

# 4. Working with arrays

In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 1000)

import pyspark.sql.functions as F
import pyspark.sql.types as T

import numpy as np
import pandas as pd

from sspipe import p, px

#### Create a dataframe with array-type columns

In [2]:
data = [
 ("James Smith",["Java","Scala","C++", "R"],["Spark","Java"],"OH","CA"),
 ("Michael Rose",["Spark","Java","C++", "Python", "Ruby"],["Spark","Java"],"NY","NJ"),
 ("Robert Williams",["CSharp","VB", "Python", "R", "Julia"],["Spark","Python"],"UT","NV")
]

from pyspark.sql.types import StringType, ArrayType,StructType,StructField
schema = StructType([ 
    StructField("name",StringType(),True), 
    StructField("languagesAtSchool",ArrayType(StringType()),True), 
    StructField("languagesAtWork",ArrayType(StringType()),True), 
    StructField("currentState", StringType(), True), 
    StructField("previousState", StringType(), True)
  ])

df = spark.createDataFrame(data=data,schema=schema)
df

name,languagesAtSchool,languagesAtWork,currentState,previousState
James Smith,"[Java, Scala, C++, R]","[Spark, Java]",OH,CA
Michael Rose,"[Spark, Java, C++, Python, Ruby]","[Spark, Java]",NY,NJ
Robert Williams,"[CSharp, VB, Python, R, Julia]","[Spark, Python]",UT,NV


#### explode: transform array to rows

In [3]:
df_explode = df.select('name', F.explode('languagesAtSchool').alias('language'))
df_explode

name,language
James Smith,Java
James Smith,Scala
James Smith,C++
James Smith,R
Michael Rose,Spark
Michael Rose,Java
Michael Rose,C++
Michael Rose,Python
Michael Rose,Ruby
Robert Williams,CSharp


#### groupby + collect_list: reverse transformation of explode, but with a random order

In [4]:
df_explode.groupby('name').agg(F.collect_list('language').alias('languages'))

name,languages
James Smith,"[Java, Scala, C++, R]"
Robert Williams,"[CSharp, VB, Python, R, Julia]"
Michael Rose,"[Spark, Java, C++, Python, Ruby]"


In [5]:
df_explode\
    .withColumn('rank', F.expr('ROW_NUMBER() OVER (PARTITION BY name ORDER BY language ASC)'))\
    .groupBy('name').agg(F.collect_list(F.struct('language', 'rank')).alias('k'))\
    .withColumn('k', F.sort_array('k'))\
    .withColumn('language', F.col('k.language'))

name,k,language
James Smith,"[{C++, 1}, {Java, 2}, {R, 3}, {Scala, 4}]","[C++, Java, R, Scala]"
Robert Williams,"[{CSharp, 1}, {Julia, 2}, {Python, 3}, {R, 4}, {VB, 5}]","[CSharp, Julia, Python, R, VB]"
Michael Rose,"[{C++, 1}, {Java, 2}, {Python, 3}, {Ruby, 4}, {Spark, 5}]","[C++, Java, Python, Ruby, Spark]"


#### Split a column into array

In [6]:
df_split = df.select(F.split('name', ' ').alias('name_array'))
df_split

name_array
"[James, Smith]"
"[Michael, Rose]"
"[Robert, Williams]"


#### Reverse splitting, using array join and regex

In [13]:
df_split.select(F.array_join('name_array', '_'))

"array_join(name_array, _)"
James_Smith
Michael_Rose
Robert_Williams


In [7]:
df_split\
    .selectExpr('CAST (name_array AS string)')\
    .select(F.regexp_replace('name_array', r'\[|\]|,', '').alias('name'))

name
James Smith
Michael Rose
Robert Williams


#### Randomly select n elements

In [86]:
df

name,languagesAtSchool,languagesAtWork,currentState,previousState
James Smith,"[Java, Scala, C++, R]","[Spark, Java]",OH,CA
Michael Rose,"[Spark, Java, C++, Python, Ruby]","[Spark, Java]",NY,NJ
Robert Williams,"[CSharp, VB, Python, R, Julia]","[Spark, Python]",UT,NV


In [35]:
df\
    .selectExpr('name', 'SLICE(SHUFFLE(languagesAtSchool), 1, 2) AS random_languages')\
    .withColumn('rank', F.expr('ROW_NUMBER() OVER (ORDER BY name)'))

name,random_languages,rank
James Smith,"[C++, Scala]",1
Michael Rose,"[Python, C++]",2
Robert Williams,"[VB, Julia]",3


In [36]:
df\
    .withColumn('rank', F.expr('ROW_NUMBER() OVER (ORDER BY name)'))\
    .selectExpr('name', 'SLICE(SHUFFLE(languagesAtSchool), 1, 2) AS random_languages', 'rank')\
    .withColumn('language', F.explode('random_languages'))

name,random_languages,rank,language
James Smith,"[C++, R]",1,C++
James Smith,"[C++, R]",1,R
Michael Rose,"[C++, Python]",2,C++
Michael Rose,"[C++, Python]",2,Python
Robert Williams,"[R, VB]",3,R
Robert Williams,"[R, VB]",3,VB


#### Concatenate two arrays

In [21]:
df.select(F.array_union('languagesAtSchool', 'languagesAtWork').alias('union'))

union
"[Java, Scala, C++, R, Spark]"
"[Spark, Java, C++, Python, Ruby]"
"[CSharp, VB, Python, R, Julia, Spark]"


In [22]:
df.select(F.array_intersect('languagesAtSchool', 'languagesAtWork').alias('intersect'))

intersect
[Java]
"[Spark, Java]"
[Python]


In [23]:
df.select(F.array_sort('languagesAtSchool').alias('sorted_array'))

"array_sort(languagesAtSchool, lambdafunction((IF(((left IS NULL) AND (right IS NULL)), 0, (IF((left IS NULL), 1, (IF((right IS NULL), -1, (IF((left < right), -1, (IF((left > right), 1, 0)))))))))), left, right))"
"[C++, Java, R, Scala]"
"[C++, Java, Python, Ruby, Spark]"
"[CSharp, Julia, Python, R, VB]"


In [2]:
data = [
 ("ABC", 'Fashion', ["Spark","Java"],"OH","CA"),
 ("MNP",["Spark","Java","C++", "Python", "Ruby"],["Spark","Java"],"NY","NJ"),
 ("XYZ",["CSharp","VB", "Python", "R", "Julia"],["Spark","Python"],"UT","NV")
]

from pyspark.sql.types import StringType, ArrayType,StructType,StructField
schema = StructType([ 
    StructField("isdn", StringType(), True), 
    StructField("category", ArrayType(StringType()), True), 
    StructField("products", ArrayType(StringType()), True), 
    StructField("category_rank", StringType(), True),
  ])

df = spark.createDataFrame(data=data,schema=schema)
df

name,languagesAtSchool,languagesAtWork,currentState,previousState
James Smith,"[Java, Scala, C++, R]","[Spark, Java]",OH,CA
Michael Rose,"[Spark, Java, C++, Python, Ruby]","[Spark, Java]",NY,NJ
Robert Williams,"[CSharp, VB, Python, R, Julia]","[Spark, Python]",UT,NV


#### Hierrachical array

In [21]:
data = (
    ('0364544993', 'headphones', 11),
    ('0364544993', 'computers', 12),
    ('0364544993', 'snacks', 13),
    ('0364544993', 'smartphones', 14),
    ('0364544993', 'makeup_men', 15),
    ('0918767457', 'kitchen', 11),
    ('0918767457', 'skin_care', 12),
    ('0918767457', 'makeup_women', 13),
    ('0918767457', 'novels', 14),
    ('0918767457', 'home_care', 15),
    ('0914934576', 'geomancy', 11),
    ('0914934576', 'worship_items', 12),
    ('0914934576', 'oral_care', 13),
    ('0914934576', 'novels', 14),
    ('0914934576', 'casual_clothes_men', 15),
)

schema = 'isdn string, category string, rank int'

df_demo = spark.createDataFrame(data, schema)
df_demo = df_demo.groupby('isdn').agg(F.collect_list(F.struct('rank', 'category')).alias('demo_recommend'))
df_demo

isdn,demo_recommend
0914934576,"[{11, geomancy}, {12, worship_items}, {13, oral_care}, {14, novels}, {15, casual_clothes_men}]"
0918767457,"[{11, kitchen}, {12, skin_care}, {13, makeup_women}, {14, novels}, {15, home_care}]"
0364544993,"[{11, headphones}, {12, computers}, {13, snacks}, {14, smartphones}, {15, makeup_men}]"


In [10]:
data = (
    ('0364544993', 'travelling', 1),
    ('0364544993', 'sexual_welness', 2),
    ('0364544993', 'computer_accessories', 3),
    ('0364544993', 'men_shoes', 4),
    ('0364544993', 'smartphones', 5),
    ('0918767457', 'kitchen', 1),
    ('0918767457', 'home_care', 2),
    ('0918767457', 'food', 3),
)

schema = 'isdn string, category string, rank int'

df_beha = spark.createDataFrame(data, schema)
df_beha = df_beha.groupby('isdn').agg(F.collect_list(F.struct('rank', 'category')).alias('beha_recommend'))
df_beha

isdn,beha_recommend
0918767457,"[{1, kitchen}, {2, home_care}, {3, food}]"
0364544993,"[{1, travelling}, {2, sexual_welness}, {3, computer_accessories}, {4, men_shoes}, {5, smartphones}]"


In [28]:
df = df_beha.join(df_demo, on='isdn', how='right')
df

isdn,beha_recommend,demo_recommend
0914934576,null,"[{11, geomancy}, {12, worship_items}, {13, oral_care}, {14, novels}, {15, casual_clothes_men}]"
0918767457,"[{1, kitchen}, {2, home_care}, {3, food}]","[{11, kitchen}, {12, skin_care}, {13, makeup_women}, {14, novels}, {15, home_care}]"
0364544993,"[{1, travelling}, {2, sexual_welness}, {3, computer_accessories}, {4, men_shoes}, {5, smartphones}]","[{11, headphones}, {12, computers}, {13, snacks}, {14, smartphones}, {15, makeup_men}]"


In [31]:
df\
    .selectExpr('isdn', 'beha_recommend.category AS category_beha', 'demo_recommend.category AS category_demo')\
    .select('isdn', F.concat('category_beha', 'category_demo').alias('recommend'))\
    .select('isdn', F.array_distinct('recommend').alias('recommend'))\
    .selectExpr('isdn', 'SLICE(recommend, 1, 5) AS recommend')\
    .selectExpr('isdn', 'POSEXPLODE(recommend)')

isdn,pos,col
0918767457,0,kitchen
0918767457,1,home_care
0918767457,2,food
0918767457,3,skin_care
0918767457,4,makeup_women
0364544993,0,travelling
0364544993,1,sexual_welness
0364544993,2,computer_accessories
0364544993,3,men_shoes
0364544993,4,smartphones


---
*&#9829; By Quang Hung x Thuy Linh &#9829;*